#Variational Quantum Eigensolver(VQE)、QAOAセミナー 2-5（組合せ最適化問題編3）

##2-5-4 実例：交通最適化
交通最適化は度々出てくるテーマです。やることは、

１、１台の車に３つのルート選択を許容する（古典）
２、現在の選択ルートから混雑状況を計算（古典）
３、混雑度を最小にするようにルート選択を最小化（QAOA）

となります。参考を確認します。

「Quantum Computing at Volkswagen:
Traffic Flow Optimization using the D-Wave Quantum Annealer」
引用：https://www.dwavesys.com/sites/default/files/VW.pdf

スタートAからゴールBまで12の道路が通し番号0から11まで付けられています。自動車１と自動車２があり、それぞれ指定されたルートを取ります。ルートはそれぞれ3つずつ提案されており、そのうちの一つを取ります。今回は一番混雑度が低くなるようなルートを選択します。

<img src="https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.ap-northeast-1.amazonaws.com%2F0%2F218694%2F2a1884aa-1fef-4e5d-94e9-810ada5b6788.png?ixlib=rb-1.2.2&auto=format&gif-q=60&q=75&w=1400&fit=max&s=2858aaec1767a1fcbb21ecfcb4644958">

一つ目は混雑度をコスト関数として実装します。上記からs0,s3,...のように、道路を指定して各出現回数を足し合わせて二乗します。上記提案ルートには、q0からq5まで通し番号を降っています。

$$
(q_0+q_1+q_3+q_4)^2+(q_0+q_1+q_3+q_4)^2+(q_0+q_3)^2+(q_0+q_3)^2+(q_1+q_4)^2+(q_1+q_2+q_4+q_5)^2+(q_2+q_5)^2+(q_2+q_5)^2+(q_2+q_5)^2\\ 
=2(q_0+q_1+q_3+q_4)^2+2(q_0+q_3)^2+(q_1+q_4)^2+(q_1+q_2+q_4+q_5)^2+3(q_2+q_5)^2\\ 
=4q_0^2+4q_0q_1+8q_0q_3+4q_0q_4+4q_1^2+2q_1q_2+4q_1q_3+8q_1q_4+2q_1q_5+4q_2^2 +2q_2q_4+8q_2q_5+4q_3^2+4q_3q_4+4q_4^2+2q_4q_5+4q_5^2\\
=4q_0+4q_0q_1+8q_0q_3+4q_0q_4+4q_1+2q_1q_2+4q_1q_3+8q_1q_4+2q_1q_5+4q_2 +2q_2q_4+8q_2q_5+4q_3+4q_3q_4+4q_4+2q_4q_5+4q_5
$$

上記は混雑度ですが、もう一つ設定する必要があります。３つのルートのうちに１つをえらぶ制約条件です。

$$
K(q_0+q_1+q_2-1)^2+K(q_3+q_4+q_5-1)^2 = -Kq_0+2Kq_0q_1+2Kq_0q_2-Kq_1+2Kq_1q_2-Kq_2-Kq_3+2Kq_3q_4+2Kq_3q_5-Kq_4+2Kq_4q_5-Kq_5+2K
$$

試しに制約条件だけを解いてみます。

In [1]:
!pip3 install blueqat

     |████████████████████████████████| 51kB 1.6MB/s 


In [2]:
from blueqat import vqe
from blueqat.pauli import qubo_bit as q

K=2
cost2 = -K*q(0)+2*K*q(0)*q(1)+2*K*q(0)*q(2)-K*q(1)+2*K*q(1)*q(2)-K*q(2)-K*q(3)+2*K*q(3)*q(4)+2*K*q(3)*q(5)-K*q(4)+2*K*q(4)*q(5)-K*q(5)
 
step = 4

result = vqe.Vqe(vqe.QaoaAnsatz(cost2, step)).run()
print(result.most_common(12))

(((1, 0, 0, 1, 0, 0), 0.08929612774285173), ((0, 1, 0, 1, 0, 0), 0.08929612774285173), ((1, 0, 0, 0, 1, 0), 0.08929612774285173), ((0, 0, 1, 0, 1, 0), 0.08929612774285173), ((0, 0, 1, 1, 0, 0), 0.0892961277428517), ((0, 1, 0, 0, 1, 0), 0.0892961277428517), ((0, 1, 0, 0, 0, 1), 0.08929612774285166), ((0, 0, 1, 0, 0, 1), 0.08929612774285166), ((1, 0, 0, 0, 0, 1), 0.08929612774285162), ((0, 0, 0, 0, 0, 1), 0.016000323795117205), ((0, 0, 1, 0, 0, 0), 0.0160003237951172), ((0, 1, 0, 0, 0, 0), 0.01600032379511716))


上手くできました。この二つを繋げてQUBO式とします。そして、実行します。

In [3]:
from blueqat import vqe
from blueqat.pauli import qubo_bit as q

K=10
cost1 = 4*q(0)+4*q(0)*q(1)+8*q(0)*q(3)+4*q(0)*q(4)+4*q(1)+2*q(1)*q(2)+4*q(1)*q(3)+8*q(1)*q(4)+2*q(1)*q(5)+4*q(2)+2*q(2)*q(4)+8*q(2)*q(5)+4*q(3)+4*q(3)*q(4)+4*q(4)+2*q(4)*q(5)+4*q(5)
cost2 = -K*q(0)+2*K*q(0)*q(1)+2*K*q(0)*q(2)-K*q(1)+2*K*q(1)*q(2)-K*q(2)-K*q(3)+2*K*q(3)*q(4)+2*K*q(3)*q(5)-K*q(4)+2*K*q(4)*q(5)-K*q(5)
 
step = 8

result = vqe.Vqe(vqe.QaoaAnsatz(cost1+cost2, step)).run()
print(result.most_common(12))

(((1, 0, 0, 1, 0, 0), 0.24117168488892132), ((0, 0, 1, 0, 0, 1), 0.13437140649191856), ((0, 0, 0, 1, 0, 0), 0.04531316000311006), ((1, 0, 0, 0, 0, 0), 0.04531316000311002), ((0, 0, 1, 0, 1, 0), 0.04024903117583349), ((0, 1, 0, 0, 0, 1), 0.04024903117583346), ((0, 0, 1, 0, 0, 0), 0.038754341983826525), ((0, 0, 0, 0, 0, 1), 0.038754341983826504), ((0, 1, 1, 0, 1, 1), 0.034975224805835314), ((0, 0, 0, 0, 1, 0), 0.027789605329907527), ((0, 1, 0, 0, 0, 0), 0.02778960532990752), ((0, 0, 0, 0, 0, 0), 0.027589638428440665))


なかなかいい答えが出ませんが、うまくいけば制約条件を満たす解を与えます。
